# Spark Structured Streaming 

`Structured Streaming` — это масштабируемая и отказоустойчивая библиотека для потоковой обработки, построенный на базе `Spark SQL`. Основная идея - с потоковыми вычислениями можно работать так же, как и со статическими данными. 

In [ ]:
import os
import random
import time
from typing import Iterator, List, Tuple

import dbldatagen as dg
import pyspark.sql.functions as F
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import window, session_window
from pyspark.sql.types import StringType

 # Если переменная окружения  `JAVA_HOME` не установлена, то тут можно её указать.
os.environ["JAVA_HOME"] = "/home/alex/java/jdk11"

Создаем сессию `Spark`, как обычно

In [ ]:
spark = SparkSession \
    .builder \
    .appName("structured") \
    .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", True) \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

Проверяем, создадим "статический" DataFrame

In [ ]:
df = spark.createDataFrame([("row1", 10), ("row2", 200)], ["column1", "columns2"])

Модель исполнения:
1. Входные данные поступают пачками (`mini-batch`) и добавляются к некоторому "бесконечному" DataFrame. Размер и частота появления `mini-batch` зависит от источника (генерируются "по триггеру").
2. Пользователем описываются некоторые операции по преобразованию "бесконечного DataFrame", как в "статическом" Spark.
3. В итоге получается "результирующий DataFrame", который является результатом работы и записывается во внешний источник (топик Kafka, консоль, файлы, etc)

Создаем Streaming DataFrame, описывая процесс получения данных из какого-нибудь источника. Поддерживается 4 встроенных источника:
- Kafka (`kafka`)
- Файлы 
- Сеть (`socket`)
- Генерация DataFrame вида `(timestamp TIMESTAMP, value LONG )`, для тестовых целей (`rate`)

In [ ]:
# Для Kafka нужно указать топик

# df = spark \
#   .readStream \
#   .format("kafka") \
#   .option("kafka.bootstrap.servers", "localhost:9092") \
#   .option("subscribePattern", "topic*") \
#   .option("startingOffsets", "earliest") \
#   .load()


# Будет создаваться 10 записей в секунду 
df = spark \
    .readStream \
    .format("rate") \
    .option("rowsPerSecond", "10") \
    .load()

df.printSchema()

Можно запустить процесс обработки, дать поработать 10 секунд и остановить. Данные накапливаются в течение некоторого времени по триггеру в так называемый `mini-batch` и обрабатываются. Затем обновления добавляются в "бесконечный" DataFrame. Режим вывода может быть:
- `update` - выводить только обновленные строки
- `complete` - DataFrame полностью
- `append` - новые строки

Не все эти режимы доступны, зависит от применяемых операций обработки DataFrame. Результат будет выводиться в консоль. 

In [ ]:
query = df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .start()

time.sleep(10)

query.stop()

Библиотека [dbldatagen](https://github.com/databrickslabs/dbldatagen) позволяет, для тестовых целей, генерировать DataFrame с заданной схемой и случайным содержимом. Создадим DataFrame с одной колонкой, в которой может быть одно из пяти заданных слов. 

In [ ]:
# описываем данные, которые будут генерироваться
ds = dg.DataGenerator(spark, name="Words", rows=20, partitions=1) \
      .withColumn("word", StringType(), values=["hello", "world", "ok", "no", "yes"], weights=[1, 1, 2, 2, 2])

# создаем Streaming DataFrame
df = ds.build(withStreaming=True, options={'rowsPerSecond': 3})

df.printSchema()

In [ ]:

query = df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .start()

time.sleep(10)

query.stop()

Теперь можно описать преобразования (подсчет слов) и выводить текущую статистику в консоль

In [ ]:

df = df.groupBy("word").count()

query = df \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()


time.sleep(30)

query.stop()

Проэмулируем получение данных от трех IoT-устройств

In [ ]:
ds = dg.DataGenerator(spark, name="IOT", rows=5000, partitions=1) \
      .withColumn("time", "timestamp", expr="now()") \
      .withColumn("sensor", StringType(), values=["sensor_1", "sensor_2", "sensor_3"]) \
      .withColumn("value", "integer", minValue=0, maxValue=10, random=True)


df = ds.build(withStreaming=True, options={'rowsPerSecond': 10})

df.printSchema()

In [ ]:
df = ds.build(withStreaming=True, options={'rowsPerSecond': 10})

df = df.groupBy("sensor").avg("value")

query = df \
    .writeStream \
    .outputMode("update") \
    .format("console") \
    .option("truncate", "false") \
    .start()

time.sleep(30)

query.stop()

Можно считать статистику по "окнам", которые образуются заданными временными интервалами

In [ ]:
df = ds.build(withStreaming=True, options={'rowsPerSecond': 10})

windowed_df = df\
    .groupBy(
        window(df.time, "10 seconds"),
        df.sensor
    ).avg("value")


query = windowed_df \
    .writeStream \
    .outputMode("update") \
    .format("console") \
    .option("truncate", "false") \
    .start()

time.sleep(60)

query.stop()

Окна могут быть накладываться друг на друга или быть привязаны к какому-то полю события ("сессионные")
.
![](https://spark.apache.org/docs/latest/img/structured-streaming-time-window-types.jpg)

In [ ]:
# Пересекающиеся - 10 секунд каждые 5.
windowed_df = df\
   .groupBy(
        window(df.time, "10 seconds", "5 seconds"),
        df.sensor) \
    .avg("value")

# Разный размер окна в зависимости от датчика
sw = session_window(df.time, \
    F.when(df.sensor == "sensor_3", "5 seconds") \
     .when(df.sensor == "sensor_2", "10 seconds") \
     .otherwise("5 seconds"))

windowed_df = df\
    .groupBy(
        sw,
        df.sensor) \
    .avg("value")

Событие может быть создано намного раньше времени физической обработки Spark'ом. Например, это может быть связано с высокой нагрузкой или проблемой с сетью. Обработка такого события приведет к изменению исторических данных. Для того, чтобы избежать подобного, можно воспользоваться "watermark", указав максимальное время между значением поля времени и временем обработки события. 

In [ ]:

windowed_df = df\
    .withWatermark("time", "15 seconds") \
    .groupBy(
        window(df.time, "10 seconds"),
        df.sensor
    ).avg("value")

`Spark Streaming` поддерживает операции вида `join` между двумя датафреймами. Причем датафрейм может быть статическим или динамическим. 

In [ ]:
df = ds.build(withStreaming=True, options={'rowsPerSecond': 10})

descr_df = spark.createDataFrame([("sensor_1", "Sensor #1"), ("sensor_2", "Sensor #2"), ("sensor_3", "Sensor #3")], ["sensor", "description"])

res_df = df.join(descr_df, "sensor")

query = res_df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .start()

time.sleep(30)

query.stop()

In [ ]:
df = ds.build(withStreaming=True, options={'rowsPerSecond': 10})

windows_df_5 = df \
    .withWatermark("time", "30 seconds") \
    .groupBy(
        window("time", "5 seconds"),
        "sensor") \
    .agg(F.avg("value").alias("value")) \
    .select(F.window_time("window").alias("time"), "value", "sensor")


windows_df_10 = df \
    .withWatermark("time", "20 seconds") \
    .groupBy(
        window("time", "10 seconds"),
        "sensor") \
    .agg(F.avg("value").alias("value")) \
    .select(F.window_time("window").alias("time"), "value", "sensor")    
 
 
res_df = windows_df_5.alias("df_5").join(
    windows_df_10.alias("df_10"),
    F.expr("""
        df_5.sensor = df_10.sensor  AND
        df_5.time >= df_10.time AND
        df_5.time <= df_10.time + interval 40 seconds
        """)
)

query = res_df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .start()

time.sleep(50)

query.stop()